In [1]:
!pip install simplemma


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

Veri seti ön işleme


In [3]:
# stop word ayıklaması yapmayan metin temizleme fonk.
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)
    text = " ".join(text.lower().strip().split())

    return " ".join(text_lemmatizer(text, lang="tr"))

In [4]:
with open("oyunlar.txt", 'r', encoding="utf-8") as dosya:

    oyun_adlari_listesi = dosya.readlines()

In [5]:
with open("hakkinda.txt", 'r', encoding="utf-8") as dosya:
    # Dosyadaki her satiri okuma ve listeye ekleme
    oyun_hak_listesi = dosya.readlines()

In [6]:
data = {
    "adi": oyun_adlari_listesi,
    "hakkinda": oyun_hak_listesi
}

indexes = [range(1,len(oyun_adlari_listesi)+1)]

df = pd.DataFrame(data=data)


In [7]:

df.head()

,adi,hakkinda
0,Counter-Strike 2\n,"Counter-Strike, 20 yıldan fazladır dünyanın he..."
1,Baldur's Gate 3\n,"Partini topla ve dostluk, ihanet, fedakârlık, ..."
2,Cyberpunk 2077\n,"Cyberpunk 2077, karanlık bir gelecekte güç, gö..."
3,Euro Truck Simulator 2\n,Önemli yüklerinizi kamyonunuzla akıl almaz mes...
4,EA SPORTS FC™ 24\n,"EA SPORTS FC™ 24, sizi Dünyanın Oyununda karşı..."


In [8]:

def clean_row(row):
    hakki = clean_text(row["hakkinda"])
    return hakki

In [9]:
clean_row(df.iloc[1])

'parti top ve dostluk ihanet fedakarlık hayat kal ve mutlak güç arzu dolu bir hikaye unut diyar a dön'

In [10]:
df["clean"] = df.apply(lambda row: clean_row(row), axis=1)
df.head()

,adi,hakkinda,clean
0,Counter-Strike 2\n,"Counter-Strike, 20 yıldan fazladır dünyanın he...",counter strike yıl fazla dünya her yer milyonl...
1,Baldur's Gate 3\n,"Partini topla ve dostluk, ihanet, fedakârlık, ...",parti top ve dostluk ihanet fedakarlık hayat k...
2,Cyberpunk 2077\n,"Cyberpunk 2077, karanlık bir gelecekte güç, gö...",cyberpunk karanlık bir gelecek güç göster ve v...
3,Euro Truck Simulator 2\n,Önemli yüklerinizi kamyonunuzla akıl almaz mes...,önem yük kamyon akıl al mesafe teslim et avrup...
4,EA SPORTS FC™ 24\n,"EA SPORTS FC™ 24, sizi Dünyanın Oyununda karşı...",ea sports fc siz dünya oyun karşıla hypermotio...


In [12]:
oyunlar = df["clean"].values
print(oyunlar[:5])

['counter strike yıl fazla dünya her yer milyonlarca oyun geri bildirim şekil üstün rekabetçi oyun deneyim sun ve şimdi cs tarih yeni bir dönem başla karşı counter strike'
 'parti top ve dostluk ihanet fedakarlık hayat kal ve mutlak güç arzu dolu bir hikaye unut diyar a dön'
 'cyberpunk karanlık bir gelecek güç göster ve vücut modifikasyon çılgın kap night city kümekent geçen bir açık dünya aksiyon macera hikaye'
 'önem yük kamyon akıl al mesafe teslim et avrupa yı bir uç uç dolaş yol kral ol ingiltere belçika almanya italya hollanda polonya ve pek çok diğer ülke keşfet düzine şehir var dayanıklılık beceri ve hız sınır zorla'
 'ea sports fc siz dünya oyun karşıla hypermotionv opta taraf optimize et oyuntarz ve geliş frostbit oyun motor ile bugün kadarki en gerçek futbol deneyim']


Sözlük (Vocab) oluşturma


In [13]:
word2idx = {}
i = 0

for oyun in oyunlar:
    for sozcuk in oyun.split():
        if sozcuk not in word2idx:
            word2idx[sozcuk] = i
            i += 1

In [14]:
word2idx

{'counter': 0,
 'strike': 1,
 'yıl': 2,
 'fazla': 3,
 'dünya': 4,
 'her': 5,
 'yer': 6,
 'milyonlarca': 7,
 'oyun': 8,
 'geri': 9,
 'bildirim': 10,
 'şekil': 11,
 'üstün': 12,
 'rekabetçi': 13,
 'deneyim': 14,
 'sun': 15,
 've': 16,
 'şimdi': 17,
 'cs': 18,
 'tarih': 19,
 'yeni': 20,
 'bir': 21,
 'dönem': 22,
 'başla': 23,
 'karşı': 24,
 'parti': 25,
 'top': 26,
 'dostluk': 27,
 'ihanet': 28,
 'fedakarlık': 29,
 'hayat': 30,
 'kal': 31,
 'mutlak': 32,
 'güç': 33,
 'arzu': 34,
 'dolu': 35,
 'hikaye': 36,
 'unut': 37,
 'diyar': 38,
 'a': 39,
 'dön': 40,
 'cyberpunk': 41,
 'karanlık': 42,
 'gelecek': 43,
 'göster': 44,
 'vücut': 45,
 'modifikasyon': 46,
 'çılgın': 47,
 'kap': 48,
 'night': 49,
 'city': 50,
 'kümekent': 51,
 'geçen': 52,
 'açık': 53,
 'aksiyon': 54,
 'macera': 55,
 'önem': 56,
 'yük': 57,
 'kamyon': 58,
 'akıl': 59,
 'al': 60,
 'mesafe': 61,
 'teslim': 62,
 'et': 63,
 'avrupa': 64,
 'yı': 65,
 'uç': 66,
 'dolaş': 67,
 'yol': 68,
 'kral': 69,
 'ol': 70,
 'ingiltere': 71,
 '

TF matrisi oluşturma


In [15]:
oyun_sayisi = len(oyunlar)
sozluk_buyuklugu = len(word2idx)

print(oyun_sayisi, sozluk_buyuklugu)

1362 5466


In [16]:
tf = np.zeros((oyun_sayisi, sozluk_buyuklugu))

for i, oyun in enumerate(oyunlar):
    sozcukler = oyun.split()
    for sozcuk in sozcukler:
        tf[i, word2idx[sozcuk]] += 1 # sözcük sayma

    tf[i] = tf[i] / len(sozcukler) # normalizasyon

print(tf[0])

[0.07142857 0.07142857 0.03571429 ... 0.         0.         0.        ]


IDF vektörü oluşturma


In [18]:
# Örnek
toy_tf = np.array([
    [0.5, 0.0, 0.0, 0.5],
    [0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.0, 0.9],
    [0.3, 0.0, 0.1, 0.6],
    [0.7, 0.0, 0.0, 0.3],
])

toy_doc_freq = np.sum(toy_tf > 0, axis=0)
toy_doc_freq

array([3, 1, 2, 5])

In [19]:
toy_idf = 5 / toy_doc_freq
print(toy_idf)
toy_idf = np.log(toy_idf)
print(toy_idf)

[1.66666667 5.         2.5        1.        ]
[0.51082562 1.60943791 0.91629073 0.        ]


In [20]:
# gerçek hesaplama kısmı
doc_freq = np.sum(tf > 0, axis=0)
print(doc_freq)

idf = np.log(oyun_sayisi / doc_freq)
print(idf)

[ 4  6 26 ...  1  1  1]
[5.83041513 5.42495002 3.95861295 ... 7.21670949 7.21670949 7.21670949]


TF-IDF matrisi oluşturma


In [24]:
tf_idf = tf * idf
print(tf_idf)

[[0.41645822 0.38749643 0.14137903 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.34365283 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.1950462  0.1950462 ]]


Benzerlik hesaplama ve sorgular


In [25]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [29]:
def oyun2str(row_index):
    oyun = df.iloc[row_index]
    adi = oyun["adi"]
    hakkinda = re.sub(r"\n+", "\n", oyun["hakkinda"]).strip()
    if len(hakkinda) > 200:
        hakkinda = hakkinda[:200] + "..."

    return f"Oyun adı: {adi} \nhakkinda: {hakkinda}"


print(oyun2str(3))

Oyun adı: Euro Truck Simulator 2
 
hakkinda: Önemli yüklerinizi kamyonunuzla akıl almaz mesafelere teslim edin, Avrupa'yı bir uçtan uca dolaşırken yolların kralı olun! İngiltere, Belçika, Almanya, İtalya, Hollanda, Polonya ve pek çok diğer ülked...


In [30]:
def benzerlerini_bul(oyun_adi, benzer_oyun_sayisi):
    oyun_indexi = df[df["adi"] == oyun_adi].index[0]
    print("Aranan Oyun:")
    print(oyun2str(oyun_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_oyun = tf_idf[oyun_indexi]
    for vec in tf_idf:
        benzerlik = cos_sim(sorgu_oyun, vec)
        benzerlikler.append(benzerlik)

    en_benzer_oyun_indexleri = (- np.array(benzerlikler)
                                ).argsort()[:benzer_oyun_sayisi + 1]
    print("Benzer Oyunlar:\n")
    for i, oyun_idx in enumerate(en_benzer_oyun_indexleri[1:]):
        print(str(i+1) + ") " +oyun2str(oyun_idx))
        print()
        print("- " * 20)
        print()

In [31]:
benzerlerini_bul("Counter-Strike 2\n", 3)

Aranan Oyun:
Oyun adı: Counter-Strike 2
 
hakkinda: Counter-Strike, 20 yıldan fazladır dünyanın her yerinden milyonlarca oyuncunun geri bildirimiyle şekillenen üstün rekabetçi oyun deneyimi sundu. Ve şimdi CS tarihinde yeni bir dönem başlıyor. Karşınız...

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Benzer Oyunlar:

1) Oyun adı: Counter-Strike: Source
 
hakkinda: Counter-Strike: Source, Counter-Strike'ın ödüllü takım oyunu aksiyonunu Source™ teknolojisinin ileri teknolojisiyle harmanlıyor.

- - - - - - - - - - - - - - - - - - - - 

2) Oyun adı: Counter-Strike Nexon: Studio
 
hakkinda: Counter-Strike Nexon: Zombies ücretsiz bir MMOFPS oyunu olup original Counter-Strike’dan içeriklerle birlikte yeni modlar, haritalar, silahlar ve Zombilerle aksiyon dolu PvP ve PvE rekabeti içerir!

- - - - - - - - - - - - - - - - - - - - 

3) Oyun adı: Left 4 Dead
 
hakkinda: Counter-Strike, Half-Life ve çok daha fazla oyunun geliştiricisi Valve tarafından geliştirilmiş Left 4 Dead; en 